In [38]:
import numpy as np
import nltk
import matplotlib.pyplot as plt
import pandas as pd

#data scaling
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split , KFold, cross_val_score, LeaveOneOut
from sklearn.naive_bayes import MultinomialNB

import spacy
import re

In [39]:
# preprocessing
data = pd.read_csv("TrainingSet1.csv")

print( data )

#temporaire pour moins de données
#data = data.head()

intentions = data["intention"].unique()
data["intention"] = data["intention"].replace({"arrosage" : np.where( intentions == 'arrosage'),
                                              "soleil": np.where( intentions == 'soleil'), 
                                              "tailler" : np.where( intentions == 'tailler'),
                                              "temperature" : np.where( intentions == 'temperature'),
                                              "cadeaux" : np.where( intentions == 'cadeaux'),
                                              "varietes" : np.where( intentions == 'varietes'),
                                              "entretien" : np.where( intentions == 'entretien'),
                                               "utilisation" : np.where( intentions == 'utilisation'),
                                               "planter" : np.where( intentions == 'planter'),
                                               "maladies" : np.where( intentions == 'maladies')
                                              })

                                         sentence intention
0    Est ce que j'ai besoin d'arroser ma plante ?  arrosage
1               Quand dois-je arroser ma plante ?  arrosage
2                     Dois-je arroser ma plante ?  arrosage
3             Comment puis-je arroser ma plante ?  arrosage
4          Ma plante a besoin de beaucoup d'eau ?  arrosage
..                                            ...       ...
210                            je suis allergique  maladies
211                          risque de maladies ?  maladies
212                  comment protéger ma plante ?  maladies
213                                   coccinelles  maladies
214            dois je utiliser des coccinelles ?  maladies

[215 rows x 2 columns]


In [40]:
y = list(data["intention"])

In [41]:
liste = data["sentence"].tolist()
j=0;

#dictionnaire des mots connus
words = list()
allowed_pos = ['VERB', 'NOUN', 'PROPN', 'ADJ']

# process sentences
for i in data["sentence"]:
    # convert all letters to lower case
    i = i.lower()
    i = i.replace('-', ' ')
    
    regex = re.compile("plante([^r]|$)")
    i = regex.sub('', i)
    
    nlp_fr = spacy.load('fr_core_news_sm')
    tokens = nlp_fr(i)

    new_sentence = ''
    for token in tokens:
        if ( allowed_pos.count(token.pos_) > 0 ):
                if (token.lemma_ != 't'): 
                    new_sentence += str(token.lemma_) + ' '
                    words.append(str(token.lemma_))
        
    liste[j] = new_sentence;
    j += 1

In [42]:
words = list(words)

In [43]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(liste).todense() #renvoie le bag of words

In [44]:
loo = LeaveOneOut()
loo.get_n_splits(X)

215

In [47]:
compt_svm = 0

for train_index, test_index in loo.split(X): 
    X_train = []
    y_train = []
    for index in train_index:
        X_train+=X[index].tolist()
        y_train.append(y[index])
    X_test = X[test_index].tolist()
    y_test = y[int(test_index)]
    
    clf_svm = svm.SVC(kernel='linear')
    clf_svm.fit(X_train, y_train)

    if( clf_svm.predict(X_test) != 'nan' ):
        if int(clf_svm.predict(X_test))==y_test:  # si prediction svm correcte
            compt_svm += 1 

In [48]:
print("Précision & Recall SVM :", compt_svm/len(X))

Précision & Recall SVM : 0.7534883720930232


In [49]:
from joblib import dump, load
dump(clf_svm, 'clf_svm.joblib')

['clf_svm.joblib']

In [50]:
# Avant de les charger à nouveau
clf_svm = load('clf_svm.joblib') 

In [53]:
# Test with a simple sentence
s = "Dois je donner de l'eau à ma plante"

# Pre-processing
s = s.lower()

regex = re.compile("plante([^r]|$)")
s = regex.sub('', s)

nlp_fr = spacy.load('fr_core_news_sm')
tokens = nlp_fr(s)

words = list()
                
# Lemmatize
for token in tokens:
    if ( allowed_pos.count(token.pos_) > 0 ):
        if (token.lemma_ != 't'): 
            words.append(str(token.lemma_))

           
j = 0;
vector = vectorizer.get_feature_names()

# Create vector
for word in vector:
    vector[j] = words.count(word);           
    j += 1



p = clf_svm.predict([vector])
print(intentions[ int(p[0]) ])

arrosage
